
# 📊 Fase 3 — Visualização e Geração de Insights (Versão Corrigida)
**Projeto:** Mineração de Textos — Análise de Sentimentos no YouTube  
**CRISP-DM:** Data Understanding / Evaluation

Este notebook atualizado faz a leitura dinâmica dos resultados da Fase 2, gera **KPIs**, **gráficos interativos** e **insights automáticos**.  
Inclui verificações automáticas, fallback de diretórios e compatibilidade com o futuro aplicativo Flask.


In [ ]:

# ============================================================
# 1) Importações e Configurações
# ============================================================
import os, json
import pandas as pd
import plotly.express as px
from glob import glob
from collections import OrderedDict

INPUT_PATTERN = "resultados/comentarios_analisados_*.csv"
EXPORT_DIR = "dashboards"
os.makedirs(EXPORT_DIR, exist_ok=True)

print("✅ Configurações prontas.")
print(f"📁 Pasta de exportação: {EXPORT_DIR}")


In [ ]:

# ============================================================
# 2) Localizar o arquivo CSV mais recente
# ============================================================
def localizar_csv(pattern="resultados/comentarios_analisados_*.csv"):
    arquivos = sorted(glob(pattern))
    if not arquivos:
        arquivos = sorted(glob("**/comentarios_analisados_*.csv", recursive=True))
        if not arquivos:
            raise FileNotFoundError("❌ Nenhum arquivo encontrado. Execute a Fase 2 antes.")
    return arquivos[-1]

INPUT_FILE = localizar_csv()
print(f"📂 Arquivo carregado: {INPUT_FILE}")

df = pd.read_csv(INPUT_FILE)
print(f"🔢 Total de registros: {len(df)}")


In [ ]:

# ============================================================
# 3) Verificar colunas essenciais
# ============================================================
colunas_esperadas = ["video_titulo", "sentimento", "idioma"]
for c in colunas_esperadas:
    if c not in df.columns:
        df[c] = "desconhecido"
        print(f"⚠️ Coluna '{c}' ausente. Valor padrão aplicado.")


In [ ]:

# ============================================================
# 4) KPIs — Métricas Principais
# ============================================================
kpi = OrderedDict()
kpi["total_comentarios"] = int(len(df))

dist = df["sentimento"].value_counts(normalize=True).reindex(["positivo","neutro","negativo"]).fillna(0.0)*100
kpi["pct_positivo"] = round(dist.get("positivo", 0.0), 2)
kpi["pct_neutro"] = round(dist.get("neutro", 0.0), 2)
kpi["pct_negativo"] = round(dist.get("negativo", 0.0), 2)

idiomas = df["idioma"].value_counts(normalize=True)*100
top5_idiomas = idiomas.head(5).round(2).to_dict()
kpi["top5_idiomas"] = top5_idiomas

por_video = df.groupby("video_titulo")["sentimento"].value_counts(normalize=True).unstack(fill_value=0.0)*100
por_video = por_video.reindex(columns=["positivo","neutro","negativo"]).fillna(0.0)

if not por_video.empty:
    kpi["melhor_video_positivo_%"] = por_video["positivo"].idxmax()
    kpi["melhor_video_positivo_val"] = round(por_video["positivo"].max(), 2)
    kpi["pior_video_negativo_%"] = por_video["negativo"].idxmax()
    kpi["pior_video_negativo_val"] = round(por_video["negativo"].max(), 2)

print("📈 KPIs Calculados:")
for k,v in kpi.items():
    print(f"- {k}: {v}")


In [ ]:

# ============================================================
# 5) Gráficos Interativos (Plotly)
# ============================================================
if not df.empty:
    # Gráfico 1 — Distribuição Geral
    fig_sent = px.bar(
        df["sentimento"].value_counts().reset_index(),
        x="index", y="sentimento",
        labels={"index":"Sentimento","sentimento":"Quantidade"},
        title="Distribuição Geral de Sentimentos"
    )
    sent_html = os.path.join(EXPORT_DIR, "01_distribuicao_sentimentos.html")
    fig_sent.write_html(sent_html, include_plotlyjs="cdn", full_html=True)
    print(f"✅ Gráfico 1 exportado: {sent_html}")

    # Gráfico 2 — Sentimentos por Vídeo
    pv = df.groupby(["video_titulo","sentimento"]).size().reset_index(name="qtd")
    if not pv.empty:
        fig_vid = px.bar(
            pv, x="video_titulo", y="qtd", color="sentimento",
            title="Sentimentos por Vídeo (Empilhado)",
            labels={"video_titulo":"Vídeo","qtd":"Quantidade"}
        )
        vid_html = os.path.join(EXPORT_DIR, "02_sentimentos_por_video.html")
        fig_vid.write_html(vid_html, include_plotlyjs="cdn", full_html=True)
        print(f"✅ Gráfico 2 exportado: {vid_html}")

    # Gráfico 3 — Top 5 Idiomas
    if not idiomas.empty:
        df_id = idiomas.head(5).reset_index()
        df_id.columns = ["idioma","percentual"]
        fig_id = px.bar(
            df_id, x="idioma", y="percentual",
            title="Top 5 Idiomas — Participação (%)",
            labels={"idioma":"Idioma","percentual":"% Comentários"}
        )
        id_html = os.path.join(EXPORT_DIR, "03_top5_idiomas.html")
        fig_id.write_html(id_html, include_plotlyjs="cdn", full_html=True)
        print(f"✅ Gráfico 3 exportado: {id_html}")
else:
    print("⚠️ Nenhum dado disponível para visualização.")


In [ ]:

# ============================================================
# 6) Insights Automáticos
# ============================================================
def gerar_insights(df, kpi):
    msgs = []
    msgs.append(f"Foram analisados {kpi['total_comentarios']} comentários.")
    msgs.append(f"Distribuição geral: {kpi['pct_positivo']}% positivos, {kpi['pct_neutro']}% neutros e {kpi['pct_negativo']}% negativos.")
    if kpi.get("melhor_video_positivo_%"):
        msgs.append(f"Vídeo mais positivo: '{kpi['melhor_video_positivo_%']}' ({kpi['melhor_video_positivo_val']}% de positividade).")
    if kpi.get("pior_video_negativo_%"):
        msgs.append(f"Vídeo com maior negatividade: '{kpi['pior_video_negativo_%']}' ({kpi['pior_video_negativo_val']}% de negatividade).")
    if kpi.get("top5_idiomas"):
        idiomas_txt = ", ".join([f"{k}: {v:.1f}%" for k, v in kpi['top5_idiomas'].items()])
        msgs.append(f"Idiomas mais frequentes: {idiomas_txt}.")
    msgs.append("Sugere-se aprofundar vídeos com alta negatividade e replicar padrões positivos.")
    return msgs

insights = gerar_insights(df, kpi)
for i in insights:
    print("💬", i)

# Salvar insights
ts = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
with open(os.path.join(EXPORT_DIR, f"insights_{ts}.txt"), "w", encoding="utf-8") as f:
    f.write("\n".join(insights))

print("\n✅ Fase 3 concluída com sucesso. Gráficos e insights exportados para 'dashboards/'.")
